# Outline
0. Create ground truth distribution and get samples
1. Parameterized models and maximum likelihood estimation
2. Calculate fourier coefficients

# 0. Ground Truth Distribution

In [1]:
from enum import Enum
class SamplingType(Enum):
    # RANDOM = 1
    # GRID = 2
    SAMPLING = 3


def gaussian_casadi_normalized(x, center, std, n=2):
    return casadi.exp(-1/2 * casadi.sum2((x - center)**2)/std)/(2*np.pi*std) # casadi.sqrt((2*np.pi*std)**n)

def gaussian_normalized(x, center, std, n=2):
    return np.exp(-1/2 * np.sum((x - center)**2)/std)/np.sqrt((2*np.pi*std)**n)


def gaussian_modes(n, num_modes, gaussian=gaussian_casadi_normalized):
    num_params = (n+1)*num_modes
    def model(x, params):
        total = 0
        for i in range(num_modes):
            center = params[i,:-1]
            std = params[i,-1]
            total += gaussian(x, center, std) 
        return total
    
    return (num_modes, n+1), model, f"{n}_mode_gaussian"

In [2]:
U_shape = (1,1)

from probability_distribution import *

x = np.array([0.5,0.5])
params = np.array([[0.25, 0.75, 0.025], [0.6, 0.3, 0.025]])

_,model,_ = gaussian_modes(2,2,gaussian=gaussian_normalized)
mu_unnorm = lambda x: model(x, params)
total = mu_total(mu_unnorm, U_shape)
mu = lambda x: mu_unnorm(x)/total

In [3]:
ground_truth = mu
distribution_name = "twin_peaks"
s_type = SamplingType.SAMPLING    
K = 5

In [5]:
from fourier_functions import *
mu_display2D(ground_truth, U_shape, f"maximum_likelihood_estimation/{distribution_name}_ground_truth")
ff = Fourier_Functions(ground_truth, U_shape, K, compute_mu=True)
ground_truth_mu_k = {k: ff[k]['mu_k'] for k in ff}


In [6]:
resolution = 100
alpha = 20 # approx how many points?
import random 

sample_x = []
sample_y = []
sample_points = []
for x in np.linspace(0, 1, resolution+1):
    for y in np.linspace(0, 1, resolution+1):
        point = np.array([x,y])
        if random.random() < ground_truth(point)*(1/resolution)**2*alpha:
            sample_x.append(x)
            sample_y.append(y)
            sample_points.append(point)

sample_values = [ground_truth(x) for x in sample_points]

In [7]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)

assert len(U_shape) == 2
X,Y = np.meshgrid(*[np.linspace(0,U_shape[i]) for i in range(2)])
_s = np.stack([X.ravel(), Y.ravel()]).T

plt.title("Ground Truth and Sampling")
ax.contourf(X,Y, np.array(list(map(mu, _s))).reshape(X.shape), cmap='binary')
ax.scatter(sample_x, sample_y)
ax.set_aspect('equal')
plt.axis('off')
plt.savefig(f"maximum_likelihood_estimation/{distribution_name}_ground_truth_sampling.pdf")
plt.close("all")

# 1. Parameterized models and maximum likelihood estimation

In [39]:
# model(x, args*)
models = [gaussian_modes(2,num_modes,gaussian=gaussian_casadi_normalized) for num_modes in range(1, 6)]


In [40]:
import casadi
import pickle
from fourier_functions import *

full_models = []


def pdf_likelihood(params, model, sample_points):
    # integral = 0
    # for x in np.linspace(0, 1, resolution+1):
    #     for y in np.linspace(0, 1, resolution+1):
    #         integral += model(casadi.DM([[x,y]]), params)/resolution**2
    l = 1
    for x in sample_points:
        l *= model(casadi.DM(x.reshape((1,2))), params)# /integral
    return l

def function_distance(params, model, sample_points, sample_values):
    l = 0 
    for i, x in enumerate(sample_points):
        l += abs(sample_values[i] - model(x, params))
    return l

for num_args, model, model_name in models:
    print(f"Model: {model_name}")
    c_opti = casadi.Opti()
    params = c_opti.variable(num_args[0],num_args[1])
    num_modes, _ = num_args
    for i in range(num_modes):
        c_opti.subject_to( params[i,-1] > 1e-5 )
        c_opti.subject_to( params[i,:-1] >= 0 )
        c_opti.subject_to( params[i,:-1] <= np.array(U_shape).reshape(1,-1) )
    c_opti.minimize(-pdf_likelihood(params, model, sample_points))
    # c_opti.minimize(function_distance(params, model, sample_points, sample_values))

    p_opts = {}
    s_opts = {'print_level': 0}
    c_opti.solver('ipopt', p_opts, s_opts)
    params_sol = c_opti.solve() 
    
    mu = lambda x: model(x, params_sol)
    mu_display2D(mu, U_shape, f"maximum_likelihood_estimation/{distribution_name}_{model_name}")
    ff = Fourier_Functions(mu, U_shape, K, compute_mu=True)
    mu_k = {k: ff[k]['mu_k'] for k in ff}
    full_models.append((params_sol, mu, mu_k))

    with open(f'maximum_likelihood_estimation/{distribution_name}_{model_name}_{K}.pkl', 'wb') as handle:
        pickle.dump(mu_k, handle, protocol=pickle.HIGHEST_PROTOCOL)


Model: 2_mode_gaussian
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_g  |   4.00us (  4.00us)   3.00us (  3.00us)         1
  nlp_grad_f  |   1.37ms (685.00us) 843.00us (421.50us)         2
   nlp_jac_g  |   9.00us (  4.50us)   8.30us (  4.15us)         2
       total  |   2.23ms (  2.23ms)   1.28ms (  1.28ms)         1


CasADi - 2022-05-29 18:45:58 WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2022-05-29 18:45:58 WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]


RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:159:
.../casadi/core/optistack_internal.cpp:999: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Invalid_Number_Detected'

In [ ]:
# plt.figure()
# cmap = plt.get_cmap('viridis')
# colors = cmap(np.linspace(0, 1, len(ff)))
# ks = [k for k in ff]

# for i, (k, color) in enumerate(zip(ks, colors), 1):
#     plt.plot(perturbation_amts, [abs(perturbation[p_amt]["c_k"][k]-agent1.c_k[k]) for p_amt in perturbation_amts], "o", label=str(k), color=color)
#     plt.plot(perturbation_amts, [min([p_amt/ff[k]["h_k"]*2*max(k), 2/ff[k]["h_k"]]) for p_amt in perturbation_amts], "-", color=color)
# plt.xlabel("Maximum Perturbation Amount")
# plt.ylabel("c_k perturbation amount")
# plt.savefig(f"perturbations/{distribution_name}_trajectory_perturbation_vs_c_k_perturbation.pdf")
# plt.close()

# plt.figure()
# for i, (k, color) in enumerate(zip(ks, colors), 1):
#     plt.plot(perturbation_amts, [abs(perturbation[p_amt]["c_k"][k]-agent1.c_k[k]) for p_amt in perturbation_amts], "o-", label=str(k), color=color)
# plt.xlabel("Maximum Perturbation Amount")
# plt.ylabel("c_k perturbation amount")
# plt.savefig(f"perturbations/{distribution_name}_trajectory_perturbation_vs_c_k_perturbation_alone.pdf")
# plt.close()

---------------------------------------------------------------------------------------------------

In [14]:
np.mean(np.array(sample_points),axis=0)

array([0.38736842, 0.58947368])

In [15]:
def one_mode(samples):
    mean = np.mean(np.array(samples),axis=0)
    std = np.sqrt(np.mean(np.array([np.linalg.norm(x-mean)**2 for x in samples])))
    return lambda x: gaussian_normalized(x, mean, std, n=2)

model = one_mode(sample_points)
model_name = "unimodal"
mu_display2D(model, U_shape, f"maximum_likelihood_estimation/{distribution_name}_{model_name}")
ff = Fourier_Functions(model, U_shape, K, compute_mu=True)
model_mu_k = {k: ff[k]['mu_k'] for k in ff}

with open(f'maximum_likelihood_estimation/{distribution_name}_{model_name}_{K}.pkl', 'wb') as handle:
    pickle.dump(model_mu_k, handle, protocol=pickle.HIGHEST_PROTOCOL)

fig = plt.figure()
ax = fig.add_subplot(111)

assert len(U_shape) == 2
X,Y = np.meshgrid(*[np.linspace(0,U_shape[i]) for i in range(2)])
_s = np.stack([X.ravel(), Y.ravel()]).T

plt.title("Unimodal with Sampling")
ax.contourf(X,Y, np.array(list(map(model, _s))).reshape(X.shape), cmap='binary')
ax.scatter(sample_x, sample_y)
ax.set_aspect('equal')
plt.axis('off')
plt.savefig(f"maximum_likelihood_estimation/{distribution_name}_{model_name}_sampling.pdf")
plt.close("all")


In [ ]:
from mpl_toolkits import mplot3d

fig = plt.figure()
ax = plt.axes(projection='3d')
x = []
y = []
z = []
for k in model_mu_k:
    x.append(k[0])
    y.append(k[1])
    z.append(abs(model_mu_k[k]-ground_truth_mu_k[k]))
ax.plot_trisurf(x, y, z, cmap='viridis', edgecolor='none')
plt.savefig(f"maximum_likelihood_estimation/{distribution_name}_{model_name}_error.pdf")